In [ ]:
!pip install spacy

In [ ]:
def top_frequent(text, num_words):
  #frequency of most common words
  import spacy
  from collections import Counter

  nlp = spacy.load("en")
  text = text
  
  #lemmatization
  doc = nlp(text)
  token_list = list()
  for token in doc:
    #print(token, token.lemma_)
    token_list.append(token.lemma_)
  token_list
  lemmatized = ''
  for _ in token_list:
    lemmatized = lemmatized + ' ' + _
  lemmatized

  #remove stopwords and punctuations
  doc = nlp(lemmatized)
  words = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
  word_freq = Counter(words)
  common_words = word_freq.most_common(num_words)
  return common_words

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from textblob import TextBlob
import pandas as pd

rest_dict = [
              {  "name"   : "the-cortez-raleigh",
                  "link"  : "https://www.yelp.com/biz/the-cortez-raleigh?osq=Restaurants&start=",
                  "pages" : 3
              },
              {  "name"   : "rosewater-kitchen-and-bar-raleigh",
                  "link"  : "https://www.yelp.com/biz/rosewater-kitchen-and-bar-raleigh?osq=Restaurants&start=",
                  "pages" : 3
              }
]

def scrape(rest_list):
  all_comment_list = list()
  for rest in rest_list:
    comment_list = list()
    for pag in range(1, rest['pages']):
      try:
        time.sleep(5)

        #URL = "https://www.yelp.com/biz/the-cortez-raleigh?osq=Restaurants&start="+str(pag*10)+"&sort_by=rating_asc"
        URL = rest['link']+str(pag*10)
        print(rest['name'], 'downloading page ', pag*10)
        page = requests.get(URL)

        #next step: parsing
        soup = BeautifulSoup(page.content, 'lxml')
        soup

        for comm in soup.find("yelp-react-root").find_all("p", {"class" : "comment__373c0__Nsutg css-n6i4z7"}):
          comment_list.append(comm.find("span").decode_contents())
          print(comm.find("span").decode_contents())
      except:
        print("could not work properly!")
    all_comment_list.append([comment_list, rest['name']])
  return all_comment_list

reviews = scrape(rest_dict)

In [ ]:
df = pd.DataFrame(reviews).explode(0)
df = df.reset_index(drop=True)
df[0:10]

In [ ]:
def perform_sentiment(x):
  testimonial = TextBlob(x)
  #testimonial.sentiment (polarity, subjectvity)
  testimonial.sentiment.polarity
  #sentiment_list.append([sentence, testimonial.sentiment.polarity, testimonial.subjectivity])
  return testimonial.sentiment.polarity

df[2] = df[0].apply(lambda x : perform_sentiment(x))
df = df.sort_values(2, ascending=False)
df

In [ ]:
text = ' '.join(list(df[0].values[0:20]))
text

In [ ]:
top_frequent(text, 100)